# This notebook serves to understand the usage of the weather API provided by Atheesan

### Import Required Packages

In [8]:
import openmeteo_requests
import requests_cache
import pandas as pd
from retry_requests import retry
import json

def show_json(obj):
    display(json.loads(obj.model_dump_json()))

### Setup the Open-Meteo API client with cache and retry on error

In [5]:
cache_session = requests_cache.CachedSession('.cache', expire_after = 3600)
retry_session = retry(cache_session, retries = 5, backoff_factor = 0.2)
openmeteo = openmeteo_requests.Client(session = retry_session)

### Define Required Weather Variables & Get API Weather Response

In [19]:
url = "https://api.open-meteo.com/v1/forecast"
params = {
	"latitude": 52.52,
	"longitude": 13.41,
	"hourly": "temperature_2m"
}

# The call below returns a python list, 
# each object (of type openmeteo_sdk.WeatherApiResponse.WeatherApiResponse) in the list 
# corresponds to weather information of a single location
responses = openmeteo.weather_api(url, params=params)

# this object contains all the weather information.
# These are the member variables:
# 

### Process first location since response is always an array

In [23]:
response = responses[0]
print(f"Coordinates {response.Latitude()}°N {response.Longitude()}°E")
print(f"Elevation {response.Elevation()} m asl")
print(f"Timezone {response.Timezone()} {response.TimezoneAbbreviation()}")
print(f"Timezone difference to GMT+0 {response.UtcOffsetSeconds()} s")

# Process hourly data. The order of variables needs to be the same as requested.
hourly = response.Hourly()
hourly_temperature_2m = hourly.Variables(0).ValuesAsNumpy()

hourly_data = {"date": pd.date_range(
	start = pd.to_datetime(hourly.Time(), unit = "s", utc = True),
	end = pd.to_datetime(hourly.TimeEnd(), unit = "s", utc = True),
	freq = pd.Timedelta(seconds = hourly.Interval()),
	inclusive = "left"
)}
hourly_data["temperature_2m"] = hourly_temperature_2m

hourly_dataframe = pd.DataFrame(data = hourly_data)
print(hourly_dataframe)

Coordinates 52.52000045776367°N 13.419998168945312°E
Elevation 38.0 m asl
Timezone None None
Timezone difference to GMT+0 0 s
                         date  temperature_2m
0   2024-04-17 00:00:00+00:00          3.2455
1   2024-04-17 01:00:00+00:00          2.6955
2   2024-04-17 02:00:00+00:00          2.3455
3   2024-04-17 03:00:00+00:00          2.2455
4   2024-04-17 04:00:00+00:00          2.0455
..                        ...             ...
163 2024-04-23 19:00:00+00:00          5.3195
164 2024-04-23 20:00:00+00:00          4.8195
165 2024-04-23 21:00:00+00:00          4.3195
166 2024-04-23 22:00:00+00:00          3.7695
167 2024-04-23 23:00:00+00:00          3.2195

[168 rows x 2 columns]
